In [ ]:
%pip install --upgrade open_spiel

In [1]:
from open_spiel.python import policy
from open_spiel.python import rl_environment
from open_spiel.python.algorithms import exploitability
from open_spiel.python.jax import nfsp
from open_spiel.python.pytorch import nfsp as nfsp_pt


class NFSPPolicies(policy.Policy):
  """Joint policy to be evaluated."""

  def __init__(self, env, nfsp_policies, mode):
    game = env.game
    player_ids = [0, 1]
    super(NFSPPolicies, self).__init__(game, player_ids)
    self._policies = nfsp_policies
    self._mode = mode
    self._obs = {"info_state": [None, None], "legal_actions": [None, None]}

  def action_probabilities(self, state, player_id=None):
    cur_player = state.current_player()
    legal_actions = state.legal_actions(cur_player)

    self._obs["current_player"] = cur_player
    self._obs["info_state"][cur_player] = state.information_state_tensor(
        cur_player
    )
    self._obs["legal_actions"][cur_player] = legal_actions

    info_state = rl_environment.TimeStep(
        observations=self._obs, rewards=None, discounts=None, step_type=None
    )

    with self._policies[cur_player].temp_mode_as(self._mode):
      p = self._policies[cur_player].step(info_state, is_evaluation=True).probs
    prob_dict = {action: p[action] for action in legal_actions}
    return prob_dict


def jax_main(
    game,
    num_players,
    env_config,
    num_train_episodes,
    eval_every,
    hidden_layers_sizes,
    replay_buffer_capacity,
    reservoir_buffer_capacity,
    anticipatory_param,
):
  env = rl_environment.Environment(game, **env_config)
  info_state_size = env.observation_spec()["info_state"][0]
  num_actions = env.action_spec()["num_actions"]

  hidden_layers_sizes = [int(ls) for ls in hidden_layers_sizes]
  kwargs = {
      "replay_buffer_capacity": replay_buffer_capacity,
      "epsilon_decay_duration": num_train_episodes,
      "epsilon_start": 0.06,
      "epsilon_end": 0.001,
  }
  expl_list = []

  # pylint: disable=g-complex-comprehension
  agents = [
      nfsp.NFSP(
          idx,
          info_state_size,
          num_actions,
          hidden_layers_sizes,
          reservoir_buffer_capacity,
          anticipatory_param,
          **kwargs
      )
      for idx in range(num_players)
  ]
  expl_policies_avg = NFSPPolicies(env, agents, nfsp.MODE.average_policy)

  for ep in range(num_train_episodes):
    if (ep + 1) % eval_every == 0:
      losses = [agent.loss for agent in agents]
      print("Losses: %s" % losses)
      expl = exploitability.exploitability(env.game, expl_policies_avg)
      expl_list.append(expl)
      print("[%s] Exploitability AVG %s" % (ep + 1, expl))
      print("_____________________________________________")

    time_step = env.reset()
    while not time_step.last():
      player_id = time_step.observations["current_player"]
      agent_output = agents[player_id].step(time_step)
      action_list = [agent_output.action]
      time_step = env.step(action_list)

    # Episode is over, step all agents with final info state.
    for agent in agents:
      agent.step(time_step)
  return expl_list


def pt_main(
    game,
    num_players,
    env_config,
    num_train_episodes,
    eval_every,
    hidden_layers_sizes,
    replay_buffer_capacity,
    reservoir_buffer_capacity,
    anticipatory_param,
):
  env = rl_environment.Environment(game, **env_config)
  info_state_size = env.observation_spec()["info_state"][0]
  num_actions = env.action_spec()["num_actions"]

  hidden_layers_sizes = [int(ls) for ls in hidden_layers_sizes]
  kwargs = {
      "replay_buffer_capacity": replay_buffer_capacity,
      "epsilon_decay_duration": num_train_episodes,
      "epsilon_start": 0.06,
      "epsilon_end": 0.001,
  }
  expl_list = []
  agents = [
      nfsp_pt.NFSP(
          idx,
          info_state_size,
          num_actions,
          hidden_layers_sizes,
          reservoir_buffer_capacity,
          anticipatory_param,
          **kwargs
      )
      for idx in range(num_players)
  ]
  expl_policies_avg = NFSPPolicies(env, agents, nfsp_pt.MODE.average_policy)
  for ep in range(num_train_episodes):
    if (ep + 1) % eval_every == 0:
      losses = [agent.loss for agent in agents]
      print("Losses: %s" % losses)
      expl = exploitability.exploitability(env.game, expl_policies_avg)
      expl_list.append(expl)
      print("[%s] Exploitability AVG %s" % (ep + 1, expl))
      print("_____________________________________________")
    time_step = env.reset()
    while not time_step.last():
      player_id = time_step.observations["current_player"]
      agent_output = agents[player_id].step(time_step)
      action_list = [agent_output.action]
      time_step = env.step(action_list)
    # Episode is over, step all agents with final info state.
    for agent in agents:
      agent.step(time_step)
  return expl_list

Optional module pokerkit_wrapper was not importable: No module named 'pokerkit'


In [2]:
game = "kuhn_poker"
num_players = 2
env_configs = {"players": num_players}
num_train_episodes = int(3e6)
eval_every = 10000
hidden_layers_sizes = [128]
replay_buffer_capacity = int(2e5)
reservoir_buffer_capacity = int(3e6)
anticipatory_param = 0.1

In [ ]:
jax_kuhn_result = jax_main(
    game,
    num_players,
    env_configs,
    num_train_episodes,
    eval_every,
    hidden_layers_sizes,
    replay_buffer_capacity,
    reservoir_buffer_capacity,
    anticipatory_param,
)

Losses: [(Array(0.56138766, dtype=float32), Array(0.36312684, dtype=float32)), (Array(0.5672982, dtype=float32), Array(0.5403075, dtype=float32))]
[10000] Exploitability AVG 0.4024593
_____________________________________________
Losses: [(Array(0.37808803, dtype=float32), Array(0.47197485, dtype=float32)), (Array(0.43798023, dtype=float32), Array(0.5559309, dtype=float32))]
[20000] Exploitability AVG 0.3080532
_____________________________________________
Losses: [(Array(0.31226516, dtype=float32), Array(0.59596515, dtype=float32)), (Array(0.29300773, dtype=float32), Array(0.45271617, dtype=float32))]
[30000] Exploitability AVG 0.27195245
_____________________________________________
Losses: [(Array(0.31338277, dtype=float32), Array(0.39556965, dtype=float32)), (Array(0.2930801, dtype=float32), Array(0.57548034, dtype=float32))]
[40000] Exploitability AVG 0.26005095
_____________________________________________
Losses: [(Array(0.30626613, dtype=float32), Array(0.45325765, dtype=float3

In [ ]:
pt_kuhn_result = pt_main(
    game,
    num_players,
    env_configs,
    num_train_episodes,
    eval_every,
    hidden_layers_sizes,
    replay_buffer_capacity,
    reservoir_buffer_capacity,
    anticipatory_param,
)

In [ ]:
import matplotlib.pyplot as plt

x = [i * 1000 for i in range(len(jax_kuhn_result))]

plt.plot(x, jax_kuhn_result, label='flax.nnx')
plt.plot(x, pt_kuhn_result, label='pytorch')
plt.title('Kuhn Poker')
plt.xlabel('Episodes')
plt.ylabel('Exploitability')
plt.legend()
plt.show()

In [ ]:
game = "leduc_poker"
num_players = 2
env_configs = {"players": num_players}
num_train_episodes = int(2e6)
eval_every = 100000
hidden_layers_sizes = [128]
replay_buffer_capacity = int(2e5)
reservoir_buffer_capacity = int(2e6)
anticipatory_param = 0.1

In [ ]:
jax_leduc_result = jax_main(
    game,
    num_players,
    env_configs,
    num_train_episodes,
    eval_every,
    hidden_layers_sizes,
    replay_buffer_capacity,
    reservoir_buffer_capacity,
    anticipatory_param,
)

In [ ]:
pt_leduc_result = pt_main(
    game,
    num_players,
    env_configs,
    num_train_episodes,
    eval_every,
    hidden_layers_sizes,
    replay_buffer_capacity,
    reservoir_buffer_capacity,
    anticipatory_param,
)

In [ ]:
x = [i * 10000 for i in range(len(jax_leduc_result))]

plt.plot(x, jax_leduc_result, label='flax.nnx')
plt.plot(x, pt_leduc_result, label='pytorch')
plt.title('Leduc Poker')
plt.xlabel('Episodes')
plt.ylabel('Exploitability')
plt.legend()
plt.show()